# Training GNNs to detect patient 0

## Imports

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import networkx as nx
import ndlib.models.epidemics as ep
import ndlib.models.ModelConfig as mc
import ndlib
import numpy as np

import plotly.graph_objects as go

## Training Data

In [181]:
NUM_NODES = 50
NUM_EDGES = 100
NUM_TRAINING = 10000
BETA = 0.15
GAMMA = 0
TRAINING_SPLIT = 0.7
def generate_data(NUM_P0s, NUM_ITERATIONS, SNAPSHOT_ARRAY):
    # Generate 50 graphs, each with 50 nodes
    NUM_P0s = 1

    graphs = np.zeros((NUM_TRAINING, NUM_NODES, NUM_NODES))
    nodes_statuses = np.zeros((NUM_TRAINING, NUM_NODES, len(SNAPSHOT_ARRAY)))
    initial_infected = []
    for i in range(NUM_TRAINING):
        graph = nx.gnm_random_graph(NUM_NODES, NUM_EDGES)
        # graph = nx.maximum_branching(graph)
        p0= np.random.randint(NUM_NODES,size=NUM_P0s)
        config = mc.Configuration()
        config.add_model_initial_configuration("Infected", p0)
        config.add_model_parameter("beta", BETA)
        config.add_model_parameter("gamma", GAMMA)

        model = ep.SIRModel(graph)
        model.set_initial_status(config)

        iterations = model.iteration_bunch(NUM_ITERATIONS)
        statuses = [iteration['status'] for iteration in iterations]
        union_of_statuses = {k:0 for k in range(NUM_NODES)}
        ind = 0
        for step,status in enumerate(statuses):
            union_of_statuses.update(status)
            if step in SNAPSHOT_ARRAY:
                # print(([v for _,v in sorted(union_of_statuses.items(), key=lambda item: item[0])]))
                nodes_statuses[i,:,ind] = ([v for _,v in sorted(union_of_statuses.items(), key=lambda item: item[0])])
                ind += 1

        graphs[i,:,:] = np.array(nx.adjacency_matrix(graph).todense())  # Convert sparse matrix to dense matrix
        # nodes_statuses.append([v for _,v in sorted(union_of_statuses.items(), key=lambda item: item[0])])
        initial_infected.append([1 if i in p0 else 0 for i in range(NUM_NODES)])

    return graphs, nodes_statuses, initial_infected

graphs, nodes_statuses, initial_infected = generate_data(1, 5, [2,4])
# print(nodes_statuses[-1])

/var/folders/hp/_5r1484j457cv8ysknrcyhvr0000gn/T/ipykernel_99427/1924104420.py:37: FutureWarning:

adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.



## Graph Visualization

In [176]:
G = nx.from_numpy_matrix(graphs[-1])
# G = nx.gnm_random_graph(NUM_NODES, NUM_EDGES)
edge_x = []
edge_y = []
positions = nx.spring_layout(G)
for edge in G.edges():
    x0, y0 = positions[edge[0]]
    x1, y1 = positions[edge[1]]
    edge_x.append(x0)
    edge_x.append(x1)
    edge_x.append(None)
    edge_y.append(y0)
    edge_y.append(y1)
    edge_y.append(None)


edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines')

node_x = []
node_y = []
for node in G.nodes():
    x, y = positions[node]
    node_x.append(x)
    node_y.append(y)

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        # colorscale options
        #'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
        #'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
        #'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
        colorscale='YlGnBu',
        reversescale=True,
        color=[],
        size=[],
        # size = 10,
        colorbar=dict(
            thickness=15,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
        line=dict(
            color='Black',
            width=2
        )))

node_adjacencies = []
node_text = []
for node, adjacencies in enumerate(G.adjacency()):
    node_adjacencies.append(len(adjacencies[1]))
    node_text.append('# of connections: '+str(len(adjacencies[1])))
avg_degree = {
    0: np.average([adj for i, adj in enumerate(node_adjacencies) if not nodes_statuses[-1,i,-1]]),
    1: np.average([adj for i, adj in enumerate(node_adjacencies) if nodes_statuses[-1,i,-1]]),
}

node_trace.marker.color = node_adjacencies
node_trace.marker.size = [20 if nodes_statuses[-1,i,-1] else 10 for i in range(NUM_NODES)]
node_trace.text = node_text

fig = go.Figure(data=[edge_trace, node_trace],
            layout=go.Layout(
                title=f'Network Snapshot of Graph with {NUM_NODES} Nodes and {NUM_EDGES} Edges where there is 1 p0 and t=5',
                titlefont_size=16,
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                annotations=[ dict(
                    text=f'Average degree of nodes is {avg_degree[2]}, average infected degree is  {avg_degree[1]:2.2f}, and diameter of graph is {nx.diameter(max([G.subgraph(c).copy() for c in nx.connected_components(G)], key=len))}',
                    showarrow=False,
                    xref="paper", yref="paper",
                    x=0.005, y=-0.002 ), ],
                    
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                )
fig.show()

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.]


In [177]:
class GCNLayer(nn.Module):
    def __init__(self, in_features, out_features, use_bias=True):
        super(GCNLayer, self).__init__()
        self.weight = nn.Parameter(torch.FloatTensor(torch.zeros(size=(in_features, out_features))))
        if use_bias:
            self.bias = nn.Parameter(torch.FloatTensor(torch.zeros(size=(out_features,))))
        else:
            self.register_parameter('bias', None)

        self.initialize_weights()

    def initialize_weights(self):
        nn.init.xavier_uniform_(self.weight)
        if self.bias is not None:
            nn.init.zeros_(self.bias)

    def forward(self, x, adj):
        x = x @ self.weight
        if self.bias is not None:
            x += self.bias

        return torch.sparse.mm(adj, x)


In [178]:

class GraphConvolutionalNetwork(nn.Module):
    def __init__(self, node_features, hidden_dim, num_classes, dropout, use_bias=True):
        super(GraphConvolutionalNetwork, self).__init__()
        self.gcn_1 = GCNLayer(node_features, hidden_dim, use_bias)
        self.gcn_2 = GCNLayer(hidden_dim, hidden_dim, use_bias)
        self.gcn_3 = GCNLayer(hidden_dim, num_classes, use_bias)
        self.dropout = nn.Dropout(p=dropout)

    def initialize_weights(self):
        self.gcn_1.initialize_weights()
        self.gcn_2.initialize_weights()
        self.gcn_3.initialize_weights()

    def forward(self, x, adj):
        x = F.relu(self.gcn_1(x, adj))
        x = self.dropout(x)
        x = F.relu(self.gcn_2(x, adj))
        x = self.dropout(x)
        x = self.gcn_3(x, adj)
        return x

def train_model(model, features, adj_matrix, labels, learning_rate):
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()

    model.train()
    optimizer.zero_grad()
    outputs = model(features, adj_matrix)
    loss = criterion(outputs.flatten(), labels.float())
    loss.backward()
    optimizer.step()
    return loss.item()

In [184]:
def train_test_model(num_features, NUM_P0s):
    model = GraphConvolutionalNetwork(num_features, 16, 1, dropout=0.1)
    overall_testing_loss = []
    criterion = nn.CrossEntropyLoss()
    max_accuracy = 0
    avg_accuracy = 0
    for _ in range(11):
        epoch_loss = 0
        for i in range(int(NUM_TRAINING * TRAINING_SPLIT)):
            features = torch.tensor(nodes_statuses[i]).reshape((NUM_NODES,num_features)).float()
            adjacency_matrix = torch.tensor(graphs[i]).float()
            labels = torch.tensor(initial_infected[i])
            epoch_loss += train_model(model, features, adjacency_matrix, labels, learning_rate=0.001)


        test_loss = 0
        correct = 0
        total = 0
        model.eval()
        avg_infected_nodes = 0
        for i in range(int(NUM_TRAINING*TRAINING_SPLIT), NUM_TRAINING):
            features = torch.tensor(nodes_statuses[i]).reshape((NUM_NODES,num_features)).float()
            adjacency_matrix = torch.tensor(graphs[i]).float()
            labels = torch.tensor(initial_infected[i])
            with torch.no_grad():
                output = model(features, adjacency_matrix)
                test_loss += criterion(output.flatten(), labels.float()).item()
                predicted = torch.topk(output.flatten(), NUM_P0s).indices
                expected = torch.topk(labels.float(), NUM_P0s).indices
                total += 1
                correct += len(np.intersect1d(np.array(predicted), np.array(expected)))/NUM_P0s
                avg_infected_nodes += torch.sum(features)
        overall_testing_loss.append(test_loss)
        max_accuracy = max(max_accuracy, 100 * correct / total)
        avg_accuracy += 100 * correct / total
    return max_accuracy, avg_accuracy/11


## Grid Results

In [185]:
TIMESTEPS = [5, 10, 20]
PATIENTS = [1,2,3]

for t in TIMESTEPS:    
    for p in PATIENTS:
        for ss in [[t-1], [(t-1)//2, t-1], [(t-1)//3, 2*(t-1)//3, t-1]]:
            graphs, nodes_statuses, initial_infected = generate_data(p, t, ss)
            max_acc, avg_acc = train_test_model(len(ss), p)
            print(f'For number of days={t}, {p} Patient 0s, and snapshots at the following time steps: {ss}, the max accuracy was {max_acc: 2.2f}% and avg accuracy was {avg_acc: 2.2f}%')

/var/folders/hp/_5r1484j457cv8ysknrcyhvr0000gn/T/ipykernel_99427/1924104420.py:37: FutureWarning:

adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.



For number of days=5, 1 Patient 0s, and snapshots at the following time steps: [4], the max accuracy was  26.73% and avg accuracy was  23.50%
For number of days=5, 1 Patient 0s, and snapshots at the following time steps: [2, 4], the max accuracy was  33.57% and avg accuracy was  26.41%
For number of days=5, 1 Patient 0s, and snapshots at the following time steps: [1, 2, 4], the max accuracy was  30.43% and avg accuracy was  21.50%
For number of days=5, 2 Patient 0s, and snapshots at the following time steps: [4], the max accuracy was  22.08% and avg accuracy was  20.08%
For number of days=5, 2 Patient 0s, and snapshots at the following time steps: [2, 4], the max accuracy was  19.93% and avg accuracy was  17.48%
For number of days=5, 2 Patient 0s, and snapshots at the following time steps: [1, 2, 4], the max accuracy was  23.23% and avg accuracy was  17.25%
For number of days=5, 3 Patient 0s, and snapshots at the following time steps: [4], the max accuracy was  19.92% and avg accuracy 